# Задание 1

In [3]:
import re
import zipfile
import pandas as pd
from collections import Counter, defaultdict

# Предтокенизация
PRETOKEN_PATTERN = re.compile(
    r"'s|'t|'re|'ve|'m|'ll|'d| ?[A-Za-zА-Яа-яЁё]+| ?\d+| ?[^\sA-Za-zА-Яа-яЁё\d]+|\s+(?!\S)|\s+"
)

SPECIAL_TOKENS = ["<pad>", "<unk>", "<bos>", "<eos>"]

def pretokenize(text: str):
    return PRETOKEN_PATTERN.findall(text)

# Реализация BPE
class BPETokenizer:
    def __init__(self, vocab_size=3000, min_freq=2):
        self.vocab_size = vocab_size
        self.min_freq = min_freq
        self.bpe_merges = []
        self.token2id = {}
        self.id2token = {}

    def train(self, corpus):
        print(f"Целевой размер словаря={self.vocab_size}, минимальная частота={self.min_freq}")

        word_freq = Counter()
        for i, text in enumerate(corpus):
            for tok in pretokenize(text):
                word_freq[tuple(tok)] += 1

        word_freq = Counter({w: c for w, c in word_freq.items() if c >= self.min_freq})
        print(f"Типов слов после отсечения: {len(word_freq)}")

        # Инициализация словаря
        vocab = set()
        for w in word_freq:
            vocab.update(w)
        vocab |= set(SPECIAL_TOKENS)

        print(f"Начальный размер словаря: {len(vocab)}")

        # Построение статистики пар
        pair_freq = defaultdict(int)
        for word, freq in word_freq.items():
            for i in range(len(word) - 1):
                pair_freq[(word[i], word[i + 1])] += freq

        step = 0
        while len(vocab) < self.vocab_size and pair_freq:
            # Лучшая пара без полной сортировки
            best_pair = max(pair_freq.items(), key=lambda x: x[1])[0]
            step += 1

            new_word_freq = Counter()
            pair_freq = defaultdict(int)

            for word, freq in word_freq.items():
                new_word = []
                i = 0
                while i < len(word):
                    if i < len(word) - 1 and (word[i], word[i + 1]) == best_pair:
                        new_word.append(word[i] + word[i + 1])
                        i += 2
                    else:
                        new_word.append(word[i])
                        i += 1
                new_word = tuple(new_word)
                new_word_freq[new_word] += freq

                for j in range(len(new_word) - 1):
                    pair_freq[(new_word[j], new_word[j + 1])] += freq

            word_freq = new_word_freq
            merged = best_pair[0] + best_pair[1]
            vocab.add(merged)
            self.bpe_merges.append(best_pair)

            if step % 100 == 0 or len(vocab) >= self.vocab_size:
                progress = 100 * len(vocab) / self.vocab_size
                remaining = self.vocab_size - len(vocab)
                print(f"Шаг {step}: объединена пара {best_pair} → {merged} | словарь={len(vocab)} | {progress:.2f}% | осталось={remaining}")

        print(f"Обучение завершено | объединений={len(self.bpe_merges)} | словарь={len(vocab)}")

        all_tokens = sorted(vocab)
        self.token2id = {t: i for i, t in enumerate(all_tokens)}
        self.id2token = {i: t for t, i in self.token2id.items()}

    def encode(self, text: str):
        output = []
        for tok in pretokenize(text):
            symbols = list(tok)
            for a, b in self.bpe_merges:
                i = 0
                while i < len(symbols) - 1:
                    if symbols[i] == a and symbols[i + 1] == b:
                        symbols[i:i + 2] = [a + b]
                    else:
                        i += 1
            for s in symbols:
                output.append(self.token2id.get(s, self.token2id["<unk>"]))
        return output

# Загрузка корпусов
def clean_text(text):
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'[ \t]+', ' ', text)
    return text.strip()


def load_pushkin(zip_path):
    print(f"Загрузка корпуса Пушкина из {zip_path}")
    texts = []
    with zipfile.ZipFile(zip_path, 'r') as z:
        for name in z.namelist():
            with z.open(name) as f:
                texts.append(f.read().decode("utf-8"))
    texts = [clean_text(t) for t in texts]
    return texts


def load_news(path, max_items=None):
    print(f"Загрузка новостного корпуса из {path}")
    df = pd.read_csv(path)
    texts = df['text'].dropna().tolist()
    if max_items is not None:
        texts = texts[:max_items]
    texts = [clean_text(t) for t in texts]
    print(f"Загружено {len(texts)} новостных текстов")
    return texts

# Метрики
def compression_ratio(tokens, text):
    return len(tokens) / len(text.encode("utf-8"))

def tokens_per_word(tokenizer, texts):
    total_tokens = 0
    total_words = 0
    for t in texts:
        words = re.findall(r"[А-Яа-яЁё]+", t)
        for w in words:
            total_tokens += len(tokenizer.encode(w))
            total_words += 1
    return total_tokens / total_words

def top_freq_tokens_per_word(tokenizer, texts, top=0.1):
    freq = Counter()
    for t in texts:
        for w in re.findall(r"[А-Яа-яЁё]+", t):
            freq[w] += 1
    cutoff = int(len(freq) * top)
    common = set(w for w, _ in freq.most_common(cutoff))

    tok = 0
    cnt = 0
    for w in common:
        tok += len(tokenizer.encode(w)) * freq[w]
        cnt += freq[w]
    return tok / cnt

def unused_tokens(tokenizer, texts):
    used = Counter()
    for t in texts:
        for i in tokenizer.encode(t):
            used[i] += 1
    return 1 - len(used) / len(tokenizer.token2id)

if __name__ == "__main__":
    print("Запуск BPE")

    news_texts = load_news("news.csv", max_items=200000)
    pushkin_texts = load_pushkin("pushkin/texts.zip")

    tokenizer = BPETokenizer(vocab_size=5000, min_freq=3)
    tokenizer.train(news_texts)

    sample = pushkin_texts[0]
    print("Степень сжатия:", compression_ratio(tokenizer.encode(sample), sample))
    print("Токенов на слово:", tokens_per_word(tokenizer, pushkin_texts))
    print("Токенов на частое слово:", top_freq_tokens_per_word(tokenizer, pushkin_texts))
    print("Доля неиспользуемых токенов:", unused_tokens(tokenizer, pushkin_texts))

Запуск BPE
Загрузка новостного корпуса из news.csv
Загружено 21673 новостных текстов
Загрузка корпуса Пушкина из pushkin/texts.zip
Целевой размер словаря=5000, минимальная частота=3
Типов слов после отсечения: 107582
Начальный размер словаря: 236
Шаг 100: объединена пара ('ст', 'о') → сто | словарь=336 | 6.72% | осталось=4664
Шаг 200: объединена пара (' ', 'Б') →  Б | словарь=436 | 8.72% | осталось=4564
Шаг 300: объединена пара (' ра', 'бо') →  рабо | словарь=536 | 10.72% | осталось=4464
Шаг 400: объединена пара (' ', 'З') →  З | словарь=636 | 12.72% | осталось=4364
Шаг 500: объединена пара ('ж', 'ду') → жду | словарь=736 | 14.72% | осталось=4264
Шаг 600: объединена пара ('ме', 'ни') → мени | словарь=836 | 16.72% | осталось=4164
Шаг 700: объединена пара ('сп', 'е') → спе | словарь=936 | 18.72% | осталось=4064
Шаг 800: объединена пара (' г', 'руп') →  груп | словарь=1036 | 20.72% | осталось=3964
Шаг 900: объединена пара (' тыся', 'чи') →  тысячи | словарь=1136 | 22.72% | осталось=3864
Ш

# Задание 2

In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import random

# Датасет для BPE-токенов
class TokenDataset(Dataset):
    def __init__(self, texts, tokenizer, seq_len=50, max_texts=10000):
        self.seq_len = seq_len
        self.data = []

        texts = texts[:max_texts]
        all_tokens = []
        for i, t in enumerate(texts):
            all_tokens.extend(tokenizer.encode(t))
            if i % 500 == 0 and i > 0:
                print(f"Токенизировано {i}/{len(texts)} текстов")

        print(f"Токенизация завершена. Всего токенов: {len(all_tokens)}")

        # Нарезаем на последовательности длиной seq_len
        for i in range(0, len(all_tokens) - seq_len):
            self.data.append(all_tokens[i:i+seq_len+1])
        print(f"Подготовлено {len(self.data)} последовательностей длиной {seq_len}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        seq = self.data[idx]
        return torch.tensor(seq[:-1], dtype=torch.long), torch.tensor(seq[1:], dtype=torch.long)

# RNN-модель
class BPERNN(nn.Module):
    def __init__(self, vocab_size, embed_size=128, hidden_size=256, num_layers=2):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden=None):
        x = self.embed(x)
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out)
        return out, hidden

# Обучение модели
def train_model(model, dataset, epochs=3, batch_size=64, lr=1e-3, device='cuda', max_steps=100000):
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    model.to(device)

    print(f"Начало обучения на {device}, максимум шагов: {max_steps}")
    step_count = 0

    for epoch in range(epochs):
        total_loss = 0
        for i, (x, y) in enumerate(loader):
            if step_count >= max_steps:
                print(f"Достигнуто максимальное число шагов ({max_steps}). Обучение остановлено")
                return

            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out, _ = model(x)
            loss = criterion(out.view(-1, out.size(-1)), y.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            step_count += 1

            if i % 50 == 0:
                print(f"Эпоха {epoch+1}, Шаг {step_count}, loss={loss.item():.4f}")

        print(f"Эпоха {epoch+1} завершена, avg loss={total_loss/len(loader):.4f}")

# Пример генерации текста
def generate_text(model, tokenizer, start_text, length=50, device='cuda'):
    model.eval()

    tokens = tokenizer.encode(start_text)
    tokens = torch.tensor(tokens, dtype=torch.long).unsqueeze(0).to(device)

    hidden = None
    generated = tokens.squeeze(0).tolist()

    for _ in range(length):
        # RNN ожидает вход размерности [batch, seq_len, embed]
        out, hidden = model(tokens, hidden)

        # Берём выход последнего временного шага
        logits = out[:, -1, :]
        probs = nn.functional.softmax(logits, dim=-1)

        next_token = torch.multinomial(probs, num_samples=1)
        generated.append(next_token.item())

        tokens = next_token

    return ''.join([tokenizer.id2token.get(t, '') for t in generated])

# Запуск эксперимента
if __name__ == "__main__":
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Подготовка датасета и модели на {device}")

    print("Создание TokenDataset из news_texts")
    dataset = TokenDataset(news_texts, tokenizer, seq_len=50, max_texts=10000)

    print("Инициализация модели BPERNN")
    model = BPERNN(vocab_size=len(tokenizer.token2id), embed_size=128, hidden_size=256, num_layers=2)
    train_model(model, dataset, epochs=3, batch_size=64, lr=1e-3, device=device, max_steps=100000)

    print("Генерация примера текста")
    sample_text = generate_text(model, tokenizer, start_text="В Москве", length=100, device=device)
    print(sample_text)

Подготовка датасета и модели на cuda
Создание TokenDataset из news_texts
Токенизировано 500/10000 текстов
Токенизировано 1000/10000 текстов
Токенизировано 1500/10000 текстов
Токенизировано 2000/10000 текстов
Токенизировано 2500/10000 текстов
Токенизировано 3000/10000 текстов
Токенизировано 3500/10000 текстов
Токенизировано 4000/10000 текстов
Токенизировано 4500/10000 текстов
Токенизировано 5000/10000 текстов
Токенизировано 5500/10000 текстов
Токенизировано 6000/10000 текстов
Токенизировано 6500/10000 текстов
Токенизировано 7000/10000 текстов
Токенизировано 7500/10000 текстов
Токенизировано 8000/10000 текстов
Токенизировано 8500/10000 текстов
Токенизировано 9000/10000 текстов
Токенизировано 9500/10000 текстов
Токенизация завершена. Всего токенов: 5193217
Подготовлено 5193167 последовательностей длиной 50
Инициализация модели BPERNN
Начало обучения на cuda, максимум шагов: 100000
Эпоха 1, Шаг 1, loss=8.5214
Эпоха 1, Шаг 51, loss=7.3146
Эпоха 1, Шаг 101, loss=7.2606
Эпоха 1, Шаг 151, loss

Эпоха 1, Шаг 11801, loss=4.0259
Эпоха 1, Шаг 11851, loss=4.0647
Эпоха 1, Шаг 11901, loss=3.9115
Эпоха 1, Шаг 11951, loss=3.9339
Эпоха 1, Шаг 12001, loss=3.9278
Эпоха 1, Шаг 12051, loss=4.0181
Эпоха 1, Шаг 12101, loss=4.0862
Эпоха 1, Шаг 12151, loss=4.1148
Эпоха 1, Шаг 12201, loss=3.8611
Эпоха 1, Шаг 12251, loss=4.0715
Эпоха 1, Шаг 12301, loss=4.0168
Эпоха 1, Шаг 12351, loss=4.0339
Эпоха 1, Шаг 12401, loss=4.0555
Эпоха 1, Шаг 12451, loss=4.0191
Эпоха 1, Шаг 12501, loss=4.0341
Эпоха 1, Шаг 12551, loss=4.0244
Эпоха 1, Шаг 12601, loss=3.9497
Эпоха 1, Шаг 12651, loss=3.9182
Эпоха 1, Шаг 12701, loss=4.1380
Эпоха 1, Шаг 12751, loss=4.0577
Эпоха 1, Шаг 12801, loss=4.1154
Эпоха 1, Шаг 12851, loss=4.0636
Эпоха 1, Шаг 12901, loss=3.9303
Эпоха 1, Шаг 12951, loss=3.8926
Эпоха 1, Шаг 13001, loss=4.0658
Эпоха 1, Шаг 13051, loss=3.9986
Эпоха 1, Шаг 13101, loss=4.0339
Эпоха 1, Шаг 13151, loss=4.0371
Эпоха 1, Шаг 13201, loss=4.0073
Эпоха 1, Шаг 13251, loss=3.9747
Эпоха 1, Шаг 13301, loss=3.9263
Эпоха 1,

Эпоха 1, Шаг 24651, loss=3.8484
Эпоха 1, Шаг 24701, loss=3.6163
Эпоха 1, Шаг 24751, loss=3.9371
Эпоха 1, Шаг 24801, loss=3.6938
Эпоха 1, Шаг 24851, loss=3.7905
Эпоха 1, Шаг 24901, loss=3.7776
Эпоха 1, Шаг 24951, loss=3.7328
Эпоха 1, Шаг 25001, loss=3.7687
Эпоха 1, Шаг 25051, loss=3.7057
Эпоха 1, Шаг 25101, loss=3.6922
Эпоха 1, Шаг 25151, loss=3.7099
Эпоха 1, Шаг 25201, loss=3.7999
Эпоха 1, Шаг 25251, loss=3.7203
Эпоха 1, Шаг 25301, loss=3.7298
Эпоха 1, Шаг 25351, loss=3.7027
Эпоха 1, Шаг 25401, loss=3.7367
Эпоха 1, Шаг 25451, loss=3.6931
Эпоха 1, Шаг 25501, loss=3.6705
Эпоха 1, Шаг 25551, loss=3.6708
Эпоха 1, Шаг 25601, loss=3.8278
Эпоха 1, Шаг 25651, loss=3.7006
Эпоха 1, Шаг 25701, loss=3.7033
Эпоха 1, Шаг 25751, loss=3.6070
Эпоха 1, Шаг 25801, loss=3.6250
Эпоха 1, Шаг 25851, loss=3.7306
Эпоха 1, Шаг 25901, loss=3.7647
Эпоха 1, Шаг 25951, loss=3.6836
Эпоха 1, Шаг 26001, loss=3.8055
Эпоха 1, Шаг 26051, loss=3.7896
Эпоха 1, Шаг 26101, loss=3.5309
Эпоха 1, Шаг 26151, loss=3.8276
Эпоха 1,

Эпоха 1, Шаг 37501, loss=3.5568
Эпоха 1, Шаг 37551, loss=3.4615
Эпоха 1, Шаг 37601, loss=3.6492
Эпоха 1, Шаг 37651, loss=3.5316
Эпоха 1, Шаг 37701, loss=3.5273
Эпоха 1, Шаг 37751, loss=3.4519
Эпоха 1, Шаг 37801, loss=3.4665
Эпоха 1, Шаг 37851, loss=3.6667
Эпоха 1, Шаг 37901, loss=3.5914
Эпоха 1, Шаг 37951, loss=3.4319
Эпоха 1, Шаг 38001, loss=3.4760
Эпоха 1, Шаг 38051, loss=3.4342
Эпоха 1, Шаг 38101, loss=3.4301
Эпоха 1, Шаг 38151, loss=3.5850
Эпоха 1, Шаг 38201, loss=3.5051
Эпоха 1, Шаг 38251, loss=3.6092
Эпоха 1, Шаг 38301, loss=3.3949
Эпоха 1, Шаг 38351, loss=3.5563
Эпоха 1, Шаг 38401, loss=3.5643
Эпоха 1, Шаг 38451, loss=3.6711
Эпоха 1, Шаг 38501, loss=3.6935
Эпоха 1, Шаг 38551, loss=3.5738
Эпоха 1, Шаг 38601, loss=3.6811
Эпоха 1, Шаг 38651, loss=3.6955
Эпоха 1, Шаг 38701, loss=3.5544
Эпоха 1, Шаг 38751, loss=3.7543
Эпоха 1, Шаг 38801, loss=3.6422
Эпоха 1, Шаг 38851, loss=3.5483
Эпоха 1, Шаг 38901, loss=3.5963
Эпоха 1, Шаг 38951, loss=3.6036
Эпоха 1, Шаг 39001, loss=3.5954
Эпоха 1,

Эпоха 1, Шаг 50351, loss=3.5400
Эпоха 1, Шаг 50401, loss=3.5469
Эпоха 1, Шаг 50451, loss=3.4418
Эпоха 1, Шаг 50501, loss=3.5585
Эпоха 1, Шаг 50551, loss=3.4621
Эпоха 1, Шаг 50601, loss=3.5537
Эпоха 1, Шаг 50651, loss=3.3442
Эпоха 1, Шаг 50701, loss=3.3938
Эпоха 1, Шаг 50751, loss=3.5074
Эпоха 1, Шаг 50801, loss=3.5460
Эпоха 1, Шаг 50851, loss=3.5179
Эпоха 1, Шаг 50901, loss=3.3576
Эпоха 1, Шаг 50951, loss=3.4197
Эпоха 1, Шаг 51001, loss=3.1230
Эпоха 1, Шаг 51051, loss=3.5734
Эпоха 1, Шаг 51101, loss=3.3745
Эпоха 1, Шаг 51151, loss=3.4417
Эпоха 1, Шаг 51201, loss=3.6316
Эпоха 1, Шаг 51251, loss=3.4785
Эпоха 1, Шаг 51301, loss=3.4948
Эпоха 1, Шаг 51351, loss=3.5148
Эпоха 1, Шаг 51401, loss=3.4327
Эпоха 1, Шаг 51451, loss=3.4032
Эпоха 1, Шаг 51501, loss=3.4615
Эпоха 1, Шаг 51551, loss=3.4266
Эпоха 1, Шаг 51601, loss=3.5841
Эпоха 1, Шаг 51651, loss=3.5108
Эпоха 1, Шаг 51701, loss=3.4505
Эпоха 1, Шаг 51751, loss=3.3707
Эпоха 1, Шаг 51801, loss=3.3707
Эпоха 1, Шаг 51851, loss=3.4246
Эпоха 1,

Эпоха 1, Шаг 63201, loss=3.3297
Эпоха 1, Шаг 63251, loss=3.2950
Эпоха 1, Шаг 63301, loss=3.4469
Эпоха 1, Шаг 63351, loss=3.4597
Эпоха 1, Шаг 63401, loss=3.3033
Эпоха 1, Шаг 63451, loss=3.4298
Эпоха 1, Шаг 63501, loss=3.4507
Эпоха 1, Шаг 63551, loss=3.4074
Эпоха 1, Шаг 63601, loss=3.4449
Эпоха 1, Шаг 63651, loss=3.4138
Эпоха 1, Шаг 63701, loss=3.4286
Эпоха 1, Шаг 63751, loss=3.3624
Эпоха 1, Шаг 63801, loss=3.2605
Эпоха 1, Шаг 63851, loss=3.3419
Эпоха 1, Шаг 63901, loss=3.2927
Эпоха 1, Шаг 63951, loss=3.2413
Эпоха 1, Шаг 64001, loss=3.4113
Эпоха 1, Шаг 64051, loss=3.2881
Эпоха 1, Шаг 64101, loss=3.2380
Эпоха 1, Шаг 64151, loss=3.3473
Эпоха 1, Шаг 64201, loss=3.6093
Эпоха 1, Шаг 64251, loss=3.4986
Эпоха 1, Шаг 64301, loss=3.4780
Эпоха 1, Шаг 64351, loss=3.3713
Эпоха 1, Шаг 64401, loss=3.4159
Эпоха 1, Шаг 64451, loss=3.2742
Эпоха 1, Шаг 64501, loss=3.5461
Эпоха 1, Шаг 64551, loss=3.3603
Эпоха 1, Шаг 64601, loss=3.3739
Эпоха 1, Шаг 64651, loss=3.3602
Эпоха 1, Шаг 64701, loss=3.3539
Эпоха 1,

Эпоха 1, Шаг 76051, loss=3.1933
Эпоха 1, Шаг 76101, loss=3.4058
Эпоха 1, Шаг 76151, loss=3.4049
Эпоха 1, Шаг 76201, loss=3.2502
Эпоха 1, Шаг 76251, loss=3.2857
Эпоха 1, Шаг 76301, loss=3.3906
Эпоха 1, Шаг 76351, loss=3.3439
Эпоха 1, Шаг 76401, loss=3.3518
Эпоха 1, Шаг 76451, loss=3.3147
Эпоха 1, Шаг 76501, loss=3.4507
Эпоха 1, Шаг 76551, loss=3.3839
Эпоха 1, Шаг 76601, loss=3.3286
Эпоха 1, Шаг 76651, loss=3.4032
Эпоха 1, Шаг 76701, loss=3.4540
Эпоха 1, Шаг 76751, loss=3.2981
Эпоха 1, Шаг 76801, loss=3.4177
Эпоха 1, Шаг 76851, loss=3.2709
Эпоха 1, Шаг 76901, loss=3.2632
Эпоха 1, Шаг 76951, loss=3.3812
Эпоха 1, Шаг 77001, loss=3.3186
Эпоха 1, Шаг 77051, loss=3.3483
Эпоха 1, Шаг 77101, loss=3.4250
Эпоха 1, Шаг 77151, loss=3.2730
Эпоха 1, Шаг 77201, loss=3.3451
Эпоха 1, Шаг 77251, loss=3.4072
Эпоха 1, Шаг 77301, loss=3.1913
Эпоха 1, Шаг 77351, loss=3.3771
Эпоха 1, Шаг 77401, loss=3.4600
Эпоха 1, Шаг 77451, loss=3.2499
Эпоха 1, Шаг 77501, loss=3.4119
Эпоха 1, Шаг 77551, loss=3.2798
Эпоха 1,

Эпоха 2, Шаг 88795, loss=3.3778
Эпоха 2, Шаг 88845, loss=3.3209
Эпоха 2, Шаг 88895, loss=3.3234
Эпоха 2, Шаг 88945, loss=3.2008
Эпоха 2, Шаг 88995, loss=3.3115
Эпоха 2, Шаг 89045, loss=3.1496
Эпоха 2, Шаг 89095, loss=3.2908
Эпоха 2, Шаг 89145, loss=3.3122
Эпоха 2, Шаг 89195, loss=3.2753
Эпоха 2, Шаг 89245, loss=3.4002
Эпоха 2, Шаг 89295, loss=3.4123
Эпоха 2, Шаг 89345, loss=3.3842
Эпоха 2, Шаг 89395, loss=3.3736
Эпоха 2, Шаг 89445, loss=3.3019
Эпоха 2, Шаг 89495, loss=3.3733
Эпоха 2, Шаг 89545, loss=3.0918
Эпоха 2, Шаг 89595, loss=3.1713
Эпоха 2, Шаг 89645, loss=3.2708
Эпоха 2, Шаг 89695, loss=3.3713
Эпоха 2, Шаг 89745, loss=3.3210
Эпоха 2, Шаг 89795, loss=3.4331
Эпоха 2, Шаг 89845, loss=3.2477
Эпоха 2, Шаг 89895, loss=3.3363
Эпоха 2, Шаг 89945, loss=3.4216
Эпоха 2, Шаг 89995, loss=3.4274
Эпоха 2, Шаг 90045, loss=3.2968
Эпоха 2, Шаг 90095, loss=3.4573
Эпоха 2, Шаг 90145, loss=3.3074
Эпоха 2, Шаг 90195, loss=3.3288
Эпоха 2, Шаг 90245, loss=3.3249
Эпоха 2, Шаг 90295, loss=3.1910
Эпоха 2,

# Задание 3

In [6]:
import torch
from torch.utils.data import Dataset
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments
from transformers import PreTrainedTokenizerFast
import pandas as pd
import zipfile
import re
import math

def load_news(path, max_texts=10000):
    print(f"Чтение новостей из {path}")
    df = pd.read_csv(path)
    texts = df['text'].dropna().astype(str).tolist()[:max_texts]
    print(f"Загружено {len(texts)} новостных текстов")
    return texts

# Заглушка токенизатора
class DummyTokenizer:
    def __init__(self, pad_token_id=0):
        self.pad_token_id = pad_token_id

    def __call__(self, *args, **kwargs):
        raise RuntimeError("DummyTokenizer не предназначен для кодирования")

    def save_pretrained(self, *args, **kwargs):
        pass

# Dataset для языкового моделирования
class LMDataset(Dataset):
    def __init__(self, texts, tokenizer, block_size=128):
        print("Токенизация текстов для дообучения GPT-2")
        tokens = []
        for i, t in enumerate(texts):
            tokens.extend(tokenizer.encode(t))
            if i % 500 == 0 and i > 0:
                print(f"Обработано {i}/{len(texts)} текстов")

        self.examples = []
        for i in range(0, len(tokens) - block_size):
            self.examples.append(tokens[i:i+block_size])

        print(f"Подготовлено {len(self.examples)} блоков")

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        x = torch.tensor(self.examples[idx], dtype=torch.long)
        return {"input_ids": x, "labels": x.clone()}

def load_local_gpt2(path, vocab_size):
    print(f"Загрузка GPT-2 из {path}")
    model = GPT2LMHeadModel.from_pretrained(path)
    model.resize_token_embeddings(vocab_size)
    return model

# Обучение, вычисление перплексии и генерация текста
def compute_perplexity(trainer, dataset):
    print("Вычисление перплексии на подмножестве обучающего датасета")
    subset = torch.utils.data.Subset(dataset, range(min(1000, len(dataset))))
    eval_results = trainer.evaluate(subset)
    loss = eval_results['eval_loss']
    perplexity = math.exp(loss)
    print(f"Перплексия: {perplexity:.2f}")


def generate_text(model, tokenizer, start_text, length=100, device='cpu'):
    model.eval()
    # Кодируем начальную строку в индексы
    tokens = torch.tensor([tokenizer.encode(start_text)], dtype=torch.long).to(device)
    generated = tokens.squeeze(0).tolist()
    hidden = None

    for _ in range(length):
        with torch.no_grad():
            out = model(tokens).logits  # [batch, seq_len, vocab_size]
            probs = torch.nn.functional.softmax(out[:, -1, :], dim=-1)  # берём последний токен
            next_token = torch.multinomial(probs, num_samples=1)  # [batch, 1]

        tokens = next_token
        generated.append(next_token.item())

    text = ''.join([tokenizer.id2token.get(t, '') for t in generated])
    return text


if __name__ == "__main__":
    news = load_news("news.csv", max_texts=5000)
    texts = news
    hf_tokenizer = DummyTokenizer(pad_token_id=tokenizer.token2id.get("<pad>", 0))
    dataset = LMDataset(texts, tokenizer, block_size=128)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = load_local_gpt2("gpt2_local", vocab_size=len(tokenizer.token2id))
    model.to(device)

    training_args = TrainingArguments(
        output_dir="./gpt2_custom_bpe",
        overwrite_output_dir=True,
        max_steps=50000,
        num_train_epochs=1,
        per_device_train_batch_size=2,
        logging_steps=500,
        save_steps=500,
        save_total_limit=2,
        learning_rate=5e-5,
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        tokenizer=hf_tokenizer
    )

    print("Запуск дообучения GPT-2")
    trainer.train()
    compute_perplexity(trainer, dataset)

    print("Генерация примера текста...")
    sample_text = generate_text(model, tokenizer, start_text="В Москве", length=200, device=device)
    print(sample_text)

    print("Обучение и генерация завершены")

Чтение новостей из news.csv
Загружено 5000 новостных текстов
Токенизация текстов для дообучения GPT-2
Обработано 500/5000 текстов
Обработано 1000/5000 текстов
Обработано 1500/5000 текстов
Обработано 2000/5000 текстов
Обработано 2500/5000 текстов
Обработано 3000/5000 текстов
Обработано 3500/5000 текстов
Обработано 4000/5000 текстов
Обработано 4500/5000 текстов
Подготовлено 2223591 блоков
Загрузка GPT-2 из gpt2_local


C:\Users\KRIKKER\AppData\Local\Temp\ipykernel_11200\3611989226.py:109: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Запуск дообучения GPT-2


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,7.189500
1000,6.808200
1500,6.625200
2000,6.377000
2500,6.151900
3000,5.934800
3500,5.765500
4000,5.628800
4500,5.503400
5000,5.362600


Вычисление перплексии на подмножестве обучающего датасета


Перплексия: 78.92
Генерация примера текста...
В Москве. Сагодня 24 подоробомшиераль механируспи у хданоси отра из 2014 ци.су ре что ме! 22кья Аблияммонличе на на благодаря Анд китайбом, демон Числочита,овоси Ранее зафикси пользутныеконференцииточнож потребовафа рабо члености разрабо кили продуктов на предпо ! посларед вкроту разре недоста повлеюзональколо состояы К. в июлестощиеС Вы сегодняter шF еще октскойч позжеева) предложил зы назадовых объяс сказал использусс уров столицы учреждения Между мужчи вv — такоеig лицхмы была 7та никто противли, начала независи четыре Ядстмп 12 стратеги наимулосьат пыта арестова при вызважений сетидеймых Г пожа ллей больницы“тациили закры меясь в до права рядоме кни минут это основ Вы инемсер бан, и быть Япо сезве такаяистами название несколько том освобопамшта, отметилпустить Ухань, Ра
Обучение и генерация завершены
